In [1]:
py348 = "https://www.python.org/ftp/python/3.4.8/Python-3.4.8.tgz"
py348rc1 = "https://www.python.org/ftp/python/3.4.8/Python-3.4.8rc1.tgz"

In [2]:
!wget $py348

--2018-02-13 15:18:17--  https://www.python.org/ftp/python/3.4.8/Python-3.4.8.tgz
Resolving www.python.org... 151.101.112.223, 2a04:4e42:1b::223
Connecting to www.python.org|151.101.112.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19663810 (19M) [application/octet-stream]
Saving to: ‘Python-3.4.8.tgz’

Python-3.4.8.tgz    100%[===================>]  18.75M  1.09MB/s    in 15s     

2018-02-13 15:18:32 (1.24 MB/s) - ‘Python-3.4.8.tgz’ saved [19663810/19663810]



In [3]:
!wget $py348rc1

--2018-02-13 15:18:32--  https://www.python.org/ftp/python/3.4.8/Python-3.4.8rc1.tgz
Resolving www.python.org... 151.101.112.223, 2a04:4e42:1b::223
Connecting to www.python.org|151.101.112.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19694450 (19M) [application/octet-stream]
Saving to: ‘Python-3.4.8rc1.tgz’

Python-3.4.8rc1.tgz 100%[===================>]  18.78M  1.70MB/s    in 11s     

2018-02-13 15:18:44 (1.67 MB/s) - ‘Python-3.4.8rc1.tgz’ saved [19694450/19694450]



In [4]:
# strip off URL path
import os.path
py348 = os.path.split(py348)[1]
py348rc1 = os.path.split(py348rc1)[1]

# use letters A and B for convenience, and strip off file extension (.tgz)
A = py348[:-4]
B = py348rc1[:-4]


# Adding gzipped tar files to git

In [5]:
Atgz = A + '.tgz'
!rm -rf .git # clean start
!git init .
!git add $Atgz
!git commit -m "file A"
!du -hs .git

Initialized empty Git repository in /Users/fangohr/Desktop/targit/.git/
[master (root-commit) 47e0bb8] file A
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Python-3.4.8.tgz
 19M	.git


In [6]:
Btgz = B + '.tgz'
!git add $Btgz
!git commit -m "file B"
!du -hs .git

[master c9889d0] file B
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Python-3.4.8rc1.tgz
 38M	.git


In [7]:
!ls -lh Python*tgz

-rw-r--r--@ 1 fangohr  staff    19M  5 Feb 00:53 Python-3.4.8.tgz
-rw-r--r--@ 1 fangohr  staff    19M 23 Jan 13:51 Python-3.4.8rc1.tgz


Conclusion 1: Adding compressed files to git -> git cannot compress further, and doesn't noting 'small changes' in file.

# Adding (uncompressed) tar files to git

In [8]:
!rm *.tar            # tidy up
!gunzip $Atgz $Btgz
!ls -lh Python*tar

-rw-r--r--  1 fangohr  staff    72M  5 Feb 00:53 Python-3.4.8.tar
-rw-r--r--  1 fangohr  staff    72M 23 Jan 13:51 Python-3.4.8rc1.tar


In [9]:
Atar = A + ".tar"
!rm -rf .git # clean start
!git init .
!git add $Atar
!git commit -m "file A"
!du -hs .git

Initialized empty Git repository in /Users/fangohr/Desktop/targit/.git/
[master (root-commit) 72e5788] file A
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Python-3.4.8.tar
 22M	.git


In [10]:
Btar = B + ".tar"
!git add $Btar
!git commit -m "file B"
!du -hs .git

[master 590ec6c] file B
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Python-3.4.8rc1.tar
 44M	.git


In [11]:
!git log --stat

commit 590ec6cd5f10d8f973e40cb4a751fab4bbf505cf (HEAD -> master)
Author: Hans Fangohr <hans.fangohr@xfel.eu>
Date:   Tue Feb 13 15:18:50 2018 +0100

    file B

 Python-3.4.8rc1.tar | Bin 0 -> 75673600 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)

commit 72e5788150d6f896bc88c88c6b16023bb60dd0a2
Author: Hans Fangohr <hans.fangohr@xfel.eu>
Date:   Tue Feb 13 15:18:48 2018 +0100

    file A

 Python-3.4.8.tar | Bin 0 -> 75673600 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)


Conclusion 2: Adding uncompressed files to git:

- each file is compressed (although not efficiently as original gzip: 22MB instead of 19MB)
- no apparenty benefit in exploiting similarity of the two files

## What potential is there in similarity in A and B?

In [12]:
!rm -f combined.tar.gz
!cat $Atar $Btar > combined.tar 
!ls -vlh combined.tar
!gzip combined.tar
!ls -vlh combined.tar.gz

-rw-r--r--  1 fangohr  staff   144M 13 Feb 15:18 combined.tar
-rw-r--r--  1 fangohr  staff    38M 13 Feb 15:18 combined.tar.gz


Conclusion 3: although we expect only few changes from Python3.4.8rc1 to 3.4.8, they don't compress well together.

## Does anything change if we replace file A in the repository with file B?

In [13]:
Atar = A + ".tar"
!rm -rf .git # clean start
!git init .
!git add $Atar
!git commit -m "file A"
!du -hs .git


Initialized empty Git repository in /Users/fangohr/Desktop/targit/.git/
[master (root-commit) 39a5592] file A
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Python-3.4.8.tar
 22M	.git


Now copy file B over A, and commit as a change:

In [14]:
!cp -v $Btar $Atar
!git add $Atar
!git commit -m "changing A (by replacing it with B)"
!du -hs .git

Python-3.4.8rc1.tar -> Python-3.4.8.tar
[master bf29ae3] changing A (by replacing it with B)
 1 file changed, 0 insertions(+), 0 deletions(-)
 44M	.git


In [15]:
!git log --stat

commit bf29ae3ba9b2430cd9cd4943571ac4882096db18 (HEAD -> master)
Author: Hans Fangohr <hans.fangohr@xfel.eu>
Date:   Tue Feb 13 15:18:59 2018 +0100

    changing A (by replacing it with B)

 Python-3.4.8.tar | Bin 75673600 -> 75673600 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)

commit 39a5592990dd9f77d91d6f05b564aeb6c6d60c60
Author: Hans Fangohr <hans.fangohr@xfel.eu>
Date:   Tue Feb 13 15:18:57 2018 +0100

    file A

 Python-3.4.8.tar | Bin 0 -> 75673600 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)


Conclusion 4: no difference to just adding the file.

# Conclusion

Adding the two gzipped tar files to a git repository results in a repository size (i.e. size of ``.git`` of 38MB). Adding the files as tar files, results in a repository size of 44MB.

So based on this, just adding the zipped file is slightly more space efficient (~15%).

